# Carga de información a la base de datos BBDD_Hoteles

En este documento realizamos la carga de los datos limpiados a la base de sql.

In [12]:
# Importamos las librerías necesarias
import pandas as pd
import numpy as np
import psycopg2 as ps

In [28]:
# Importamos el csv con la información de las reservas, los hoteles y los clientes
data = pd.read_csv("../data/reservas_hoteles_limpio.csv", parse_dates=['fecha_reserva', 'inicio_estancia', 'final_estancia'])
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15000 entries, 0 to 14999
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   id_reserva       15000 non-null  object        
 1   id_cliente       15000 non-null  object        
 2   nombre           15000 non-null  object        
 3   apellido         15000 non-null  object        
 4   mail             15000 non-null  object        
 5   competencia      15000 non-null  bool          
 6   fecha_reserva    15000 non-null  datetime64[ns]
 7   inicio_estancia  15000 non-null  datetime64[ns]
 8   final_estancia   15000 non-null  datetime64[ns]
 9   id_hotel         15000 non-null  object        
 10  nombre_hotel     15000 non-null  object        
 11  estrellas        15000 non-null  float64       
 12  ciudad           15000 non-null  object        
 13  precio_noche     15000 non-null  float64       
dtypes: bool(1), datetime64[ns](3), float64

In [13]:
# Importamos el csv con los eventos de Madrid sacados de la API
eventos = pd.read_csv("../data/eventos_madrid.csv", parse_dates=['fecha_inicio', 'fecha_fin'])
eventos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 217 entries, 0 to 216
Data columns (total 8 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   nombre_evento  217 non-null    object        
 1   url_evento     217 non-null    object        
 2   codigo_postal  217 non-null    int64         
 3   direccion      201 non-null    object        
 4   horario        134 non-null    object        
 5   organizacion   202 non-null    object        
 6   fecha_inicio   217 non-null    datetime64[ns]
 7   fecha_fin      217 non-null    datetime64[ns]
dtypes: datetime64[ns](2), int64(1), object(5)
memory usage: 13.7+ KB


## Inicialización de la conexión con la BBDD y el cursor

In [14]:
# Creamos la conexión a la base de datos
# Vamos a crear una conexión a la base de datos.
conn = ps.connect(
    dbname = "BBDD_Hoteles", # base a la que nos queremos conectar
    user = "postgres",
    password = "admin",
    host = "localhost",
    port = "5432" # puerto en el que s eencuentra postgres
)

In [15]:
# Creamos un cursor el cual nos va a permitir ejecutar querys.
cur = conn.cursor()

In [16]:
# COmprobamos que la conexión está creada y conectada
cur.execute("SELECT version();")
cur.fetchone() 

('PostgreSQL 16.4, compiled by Visual C++ build 1940, 64-bit',)

## Inserción de datos

In [17]:
# COmprobamos que no haya valores duplicados en ambos dataframes
# data = data.drop_duplicates()
eventos.duplicated().sum()

np.int64(0)

### Tabla ciudad

In [18]:
data_insert_ciudad = ["Madrid"]

In [19]:
# Creamos la query de inserción de los datos de ciudad
insert_query_ciudad = """ 
                        INSERT INTO ciudad (nombre_ciudad)
                        VALUES (%s) 
"""

In [20]:
# Ejecutamos la query y le indicamos que solo tenemos una ciudad que es Madrid
cur.execute(insert_query_ciudad, data_insert_ciudad)
conn.commit()

In [21]:
# vamos a comprobar que se ha creado correctamente
query_ciudad = """
    SELECT * 
    FROM ciudad; 
"""
cur.execute(query_ciudad)
cur.fetchall()

[(1, 'Madrid')]

# NO VOLVER A EJECUTAR LA QUERY DE CIUDAD, ESTA YA INSERTADA

### Tabla eventos

In [22]:
# Sacamos el id de ciudad de la tabla de ciudad
cur.execute("SELECT nombre_ciudad, id_ciudad FROM ciudad")
dictio_ciudad = dict(cur.fetchall())
dictio_ciudad

{'Madrid': 1}

In [23]:
# Creamos la lista que va a contener toda la información 
data_insert_eventos = []

# Vamos iterando por cada columna del dataframe y cada fila de cada columna, obteniendo los elementos de las filas (row)
for i, row in eventos.iterrows():
    nombre_evento = row["nombre_evento"]
    url_evento = row["url_evento"]
    codigo_postal = row["codigo_postal"]
    direccion = row["direccion"] if pd.notna(row["direccion"]) else None # tenemos valores nulos en esta columna, de manera que le indicamos, que si el elemento no es nulo nos lo coja, y si es nulo, nos incluya un None, ya que sql no acepta valores nulos que no sean None
    horario = row["horario"] if pd.notna(row["horario"]) else None
    fecha_inicio = row["fecha_inicio"]
    fecha_fin = row["fecha_fin"]
    organizacion = row["organizacion"] if pd.notna(row["organizacion"]) else None
    id_ciudad = dictio_ciudad.get("Madrid")

    # incluimos los valores en la lista
    data_insert_eventos.append((nombre_evento, url_evento, codigo_postal, direccion, horario, fecha_inicio, fecha_fin, organizacion, id_ciudad))

In [24]:
# Creamos la query de inserción de los datos de eventos
insert_query_eventos = """ 
                        INSERT INTO eventos (nombre_evento, url_evento, codigo_postal, direccion, horario,
                        fecha_inicio, fecha_fin,organizacion, id_ciudad)
                        VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)
"""

In [25]:
# Ejecutamos la query y le indicamos los valores 
cur.executemany(insert_query_eventos, data_insert_eventos)
conn.commit()

In [26]:
# vamos a comprobar que se ha creado correctamente
query_eventos = """
    SELECT * 
    FROM eventos; 
"""
cur.execute(query_eventos)
cur.fetchmany(3)

[(1,
  '25º aniversario de la revista La Fragua',
  'http://www.madrid.es/sites/v/index.jsp?vgnextchannel=ca9671ee4a9eb410VgnVCM100000171f5a0aRCRD&vgnextoid=1ea220bed10d4910VgnVCM2000001f4a900aRCRD',
  28005,
  'CALLE SAN JUSTO 5',
  None,
  datetime.date(2025, 2, 24),
  datetime.date(2025, 3, 2),
  'Biblioteca Pública Municipal Iván de Vargas (Centro)',
  1),
 (2,
  '60 Premio Reina Sofía de Pintura y Escultura',
  'http://www.madrid.es/sites/v/index.jsp?vgnextchannel=ca9671ee4a9eb410VgnVCM100000171f5a0aRCRD&vgnextoid=34c5130393e15910VgnVCM2000001f4a900aRCRD',
  28009,
  'PASEO COLOMBIA 1',
  None,
  datetime.date(2025, 2, 27),
  datetime.date(2025, 3, 23),
  'Centro Cultural Casa de Vacas (Retiro)',
  1),
 (3,
  'A toda vela',
  'http://www.madrid.es/sites/v/index.jsp?vgnextchannel=ca9671ee4a9eb410VgnVCM100000171f5a0aRCRD&vgnextoid=59d5e7da8b822910VgnVCM1000001d4a900aRCRD',
  28045,
  'PLAZA LEGAZPI 8',
  None,
  datetime.date(2024, 10, 1),
  datetime.date(2025, 6, 1),
  'Matadero Ma

# NO VOLVER A EJECUTAR LA QUERY DE EVENTOS, ESTA YA INSERTADA

### Tabla hoteles

In [29]:
# revisar tipo de datos
data.columns

Index(['id_reserva', 'id_cliente', 'nombre', 'apellido', 'mail', 'competencia',
       'fecha_reserva', 'inicio_estancia', 'final_estancia', 'id_hotel',
       'nombre_hotel', 'estrellas', 'ciudad', 'precio_noche'],
      dtype='object')

In [42]:
data_insert_hotels = []

for _, row in data.iterrows():
    id_hotel = row["id_hotel"]
    nombre_hotel = row["nombre_hotel"]
    competencia = row["competencia"]
    valoracion = row["estrellas"]
    id_ciudad = dictio_ciudad.get("Madrid")

    data_insert_hotels.append((id_hotel, nombre_hotel, competencia, valoracion, id_ciudad))

In [31]:
# Creamos la query de inserción de los datos de ciudad
insert_query_hoteles = """ 
                        INSERT INTO hoteles (id_hotel, nombre_hotel, competencia, valoracion, id_ciudad)
                        VALUES (%s, %s, %s, %s, %s)
"""

In [35]:
conn.rollback()

In [36]:
# Ejecutamos la query y le indicamos los valores 
cur.executemany(insert_query_hoteles, data_insert_hotels)
conn.commit()

UniqueViolation: llave duplicada viola restricción de unicidad «hoteles_pkey»
DETAIL:  Ya existe la llave (id_hotel)=(0        1
1        2
2        3
3        4
4        5
        ..
5167    28
5168    26
5169    27
5170    23
5171    21
Name: id_hotel, Length: 15000, dtype: object).


### Tabla clientes

In [30]:
# revisar tipo de datos
data.columns

Index(['id_reserva', 'id_cliente', 'nombre', 'apellido', 'mail', 'competencia',
       'fecha_reserva', 'inicio_estancia', 'final_estancia', 'id_hotel',
       'nombre_hotel', 'estrellas', 'ciudad', 'precio'],
      dtype='object')

In [42]:
data_insert_clientes = []

for _, row in data.iterrows():
    id_cliente = row["id_cliente"]
    nombre = row["nombre"],
    apellido = row["apellido"],
    mail = row["mail"]

    data_insert_clientes.append((id_cliente, nombre, apellido, mail))

In [43]:
# Creamos la query de inserción de los datos de ciudad
insert_query_clientes = """ 
                        INSERT INTO clientes (id_cliente, nombre, apellido, mail)
                        VALUES (%s, %s, %s, %s)
"""

In [41]:
conn.rollback()

In [44]:
# Ejecutamos la query y le indicamos los valores 
cur.executemany(insert_query_clientes, data_insert_clientes)
conn.commit()

UniqueViolation: llave duplicada viola restricción de unicidad «clientes_pkey»
DETAIL:  Ya existe la llave (id_cliente)=(205).


### Tabla reservas

In [ ]:
# revisar tipo de datos
data.columns

Index(['id_reserva', 'id_cliente', 'nombre', 'apellido', 'mail', 'competencia',
       'fecha_reserva', 'inicio_estancia', 'final_estancia', 'id_hotel',
       'precio_noche', 'nombre_hotel', 'estrellas', 'ciudad'],
      dtype='object')

In [ ]:
# Sacamos el id de clientes de la tabla de clientes
cur.execute("SELECT cliente, id_cliente FROM clientes")
clientes = dict(cur.fetchall())
clientes

In [ ]:
# Sacamos el id de hotel de la tabla de hoteles
cur.execute("SELECT hotel, id_hotel FROM hoteles")
hoteles = dict(cur.fetchall())
hoteles

In [ ]:
data_insert_reservas = []

for _, row in data.iterrows():
    id_reserva = row["id_reserva"]
    fecha_reserva = row["fecha_reserva"],
    inicio_estancia = row["inicio_estancia"],
    final_estancia = row["final_estancia"],
    precio_noche = data["precio_noche"],
    id_cliente = clientes.get(row["id_cliente"]),
    id_hotel = hoteles.get(row["id_hotel"])

    data_insert_eventos.append(id_reserva, fecha_reserva, inicio_estancia, final_estancia, precio_noche, id_cliente, id_hotel)

In [ ]:
# Creamos la query de inserción de los datos de ciudad
insert_query_reservas = """ 
                        INSERT INTO eventos (id_reserva, fecha_reserva, inicio_estancia, final_estancia, precio_noche, id_cliente, id_hotel)
                        VALUES (%s, %s, %s, %s, %s, %s, %s)
"""

In [ ]:
# Ejecutamos la query y le indicamos los valores 
cur.execute(insert_query_reservas, data_insert_reservas)
conn.commit()

## Cierre de la conexión

In [13]:
# una vez hemos terminado de trabajar es necesario cerrar la conexión y el cursor.
cur.close()
conn.close()